In [0]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
df = spark.table("workspace.bronze.crm_cust_info")

In [0]:
df.display()

#Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
df.display()

#Normalization

In [0]:
df = df.withColumn("cst_marital_status",f.when(f.upper(f.col("cst_marital_status"))== "S", "Single")
 .when(f.upper(f.col("cst_marital_status"))== "M", "Married")
 .otherwise("n/a") 
 )


In [0]:
df.display()

In [0]:
df = df.withColumn(
        "cst_gndr",
        f.when(f.upper(f.col("cst_gndr")) == "F", "Female")
         .when(f.upper(f.col("cst_gndr")) == "M", "Male")
         .otherwise("n/a")
    )

df.display()

#Remove Records with Missing Customer ID

In [0]:

df = df.filter(col("cst_id").isNotNull())

#Renaming Columns

In [0]:

RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_number",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

#Sanity checks of dataframe

In [0]:

df.display()

#Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_customers")

#Sanity checks of silver table

In [0]:
%sql
SELECT * FROM workspace.silver.crm_customers LIMIT 10